# ECE 763 Project 01: Training Multi-Layer Perceptrons and Convoluational Neural Networks on CIFAR10


Objectives: 

- To get hands-on experiences on working with images (CIFAR10) and neural networks by implementing multi-layer perceptrons (MLPs) and convolutional neural networks (ConvNets).
- To test different regularization strategies to get the best performance out of your models.
- To investigate the limitations of using MLPs in this task, compared with ConvNets. 
- To investigate the potential risk of fitting noise of both MLPs and ConvNets.


### What to Do?

* Assume you have installed the ece763 environment. Run this notebook with that environment.
* Write your code and text for all TODOs as shown by:

```python
    # TODO: ??? points - descriptions of what to do
    raise NotImplementedError # comment it out after you write your code
```

### How to submit your solutions

* Add your NCSU ID as the postfix of the notebook filename, e.g., `Project_01_MLPs_ConvNets_twu19.ipynb`
* Submit two versions of your notebook, one is fully executed with all outputs (`Project_01_MLPs_ConvNets_twu19_results.ipynb`), and the other with all outputs cleared (`Project_01_MLPs_ConvNets_twu19_empty.ipynb`). We will re-run the latter and expect the results will be exactly the same as those in the former.

* Late policy:  5 free late days (counted using 0.5 unit, <=6 hours as 0.5 late day, otherwise 1 later day) in total – use them in your ways; Afterwards, 25% off per day late; Not accepted after 3 late days per HW and Project. Not applicable to the final project. 
* Important Note: We will NOT accept any replacement of submission after deadline, even if you can show the time stamp of the replacement is earlier than the deadline. So, please double-check if you submit correct files.
* Academic Integrity: Students are required to comply with the university policy on academic integrity found in the Code of Student Conduct found at http://policies.ncsu.edu/policy/pol-11-35-01 
* Academic Honesty: See http://policies.ncsu.edu/policy/pol-11-35-01  for a detailed explanation of academic honesty.


## Import libraries to be Used

In [ ]:
## Standard libraries
import os
import json
import math
import random
import numpy as np 
import copy
import time

## Imports for plotting
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
sns.set_theme()

## Progress bar
from tqdm.notebook import tqdm

## typing
from typing import (
    Any,
    Callable,
    Dict,
    List,
    Optional,
    Sequence,
    Set,
    Tuple,
    Type,
    Union,
)

try:
    from typing import Literal
except ImportError:
    from typing_extensions import Literal

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## PyTorch Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms

### Identify the system


In [ ]:
import platform
from mmengine.utils.dl_utils import collect_env
from pprint import pprint

print("=" * 40, "System Information", "=" * 40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

print("=" * 40, "Environment Information", "=" * 40)
my_env = collect_env()
pprint(my_env)

## Set Random Seeds and Devices

In [ ]:
# Function for setting the seed
def random_seed(seed: int = 42, rank: int = 0, deterministic: bool = False) -> None:
    # TODO: 2 points  - write your code below
    torch.manual_seed(seed+rank)
    np.random.seed(seed+rank)
    random.seed(seed+rank)
    # Seed for GPU utilization
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed + rank)
        torch.cuda.manual_seed_all(seed + rank)
        # To make all operations on GPU deterministic 
        torch.backends.cudnn.deterministic = deterministic
        torch.backends.cudnn.benchmark = True
    

seed = 42
random_seed(seed=seed, deterministic=True)

# Fetching the device that will be used throughout this notebook
device = (
    torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
)
print("Using device", device)

## Prepare Datasets (CIFAR10)

In [ ]:
# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../checkpoints/ece763_proj_01"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

### CIFAR10
CIFAR10 is a very popular dataset for computer vision on low-resolution images (32x32 pixels). The task is to classify images into one of 10 classes: **airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck**. 

In [ ]:
# Dataset statistics for normalizing the input values to zero mean and one std
DATA_MEANS = [0.491, 0.482, 0.447]
DATA_STD = [0.247, 0.243, 0.261]

# Transformations are applied on images when we want to access them. Here, we push the images into a tensor
# and normalize the values. However, you can use more transformations, like augmentations to prevent overfitting.
# Feel free to experiment with augmentations here once you have a first running MLP, but remember to not apply
# any augmentations on the test data!
data_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(DATA_MEANS, DATA_STD)
                                     ])

# Loading the training dataset. We need to split it into a training and validation part
main_dataset = CIFAR10(root=DATASET_PATH, train=True, transform=data_transforms, download=True)
train_set, val_set = torch.utils.data.random_split(main_dataset, [45000, 5000], generator=torch.Generator().manual_seed(seed))

# Loading the test set
test_set = CIFAR10(root=DATASET_PATH, train=False, transform=data_transforms, download=True)

# Create data loaders for later
train_loader = data.DataLoader(train_set, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=3)
val_loader = data.DataLoader(val_set, batch_size=128, shuffle=False, drop_last=False, num_workers=3)
test_loader = data.DataLoader(test_set, batch_size=128, shuffle=False, drop_last=False, num_workers=3)

# classes
classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
)

When working with data, it is always recommend to look at the data before blaming your model for not performing well if the data was incorrectly processed. Hence, let's plot 1-batch images of the CIFAR10 training dataset:

In [ ]:
start_time = time.time()
dataiter = iter(train_loader)
images, labels = next(dataiter)
end_time = time.time()
print(f"Time for loading a batch: {(end_time - start_time):6.5f}s")

In [ ]:
INV_DATA_MEANS = torch.tensor([-m for m in DATA_MEANS]).view(-1, 1, 1)
INV_DATA_STD = torch.tensor([1.0 / s for s in DATA_STD]).view(-1, 1, 1)

def imshow(img):
    img = img.div_(INV_DATA_STD).sub_(INV_DATA_MEANS) # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()
    plt.close()


imshow(torchvision.utils.make_grid(images))
print("GroundTruth (1st row): ", " ".join(f"{classes[labels[j]]:5s}" for j in range(8)))

Now we are all set. So, let's dive into implementing our own MLP and ConvNet!

## Part 0: Optimizer

To gain a better insight in the training of the neural networks, let's implement our own optimizer. First, we need to understand what an optimizer actually does. The optimizer is responsible to update the network's parameters given the gradients. Hence, we effectively implement a function $w^{t} = f(w^{t-1}, g^{t}, ...)$ with $w$ being the parameters, and $g^{t} = \nabla_{w^{(t-1)}} \mathcal{L}^{(t)}$ the gradients at time step $t$. A common, additional parameter to this function is the learning rate, here denoted by $\eta$. Usually, the learning rate can be seen as the "step size" of the update. A higher learning rate means that we change the weights more in the direction of the gradients, a smaller means we take shorter steps. 

As most optimizers only differ in the implementation of $f$, we can define a template for an optimizer in PyTorch below. We take as input the parameters of a model and a learning rate. The function `zero_grad` sets the gradients of all parameters to zero, which we have to do before calling `loss.backward()`. Finally, the `step()` function tells the optimizer to update all weights based on their gradients. The template is setup below:

In [ ]:
class OptimizerTemplate:
    
    def __init__(self, params: nn.ParameterList, lr: float)->None:
        self.params = list(params)
        self.lr = lr
        
    def zero_grad(self)->None:
        ## Set gradients of all parameters to zero
        for p in self.params:
            if p.grad is not None:
                p.grad.detach_() # For second-order optimizers important
                p.grad.zero_()
    
    @torch.no_grad()
    def step(self)->None:
        ## Apply update step to all parameters
        for p in self.params:
            if p.grad is None: # We skip parameters without any gradients
                continue
            self.update_param(p)
            
    def update_param(self, p: nn.Parameter)->None:
        # To be implemented in optimizer-specific classes
        raise NotImplementedError

The optimizer we are going to implement is the standard Stochastic Gradient Descent (SGD) with momentum. Plain SGD updates the parameters using the following equation:

$$
\begin{split}
    w^{(t)} & = w^{(t-1)} - \eta \cdot g^{(t)}
\end{split}
$$

The concept of momentum replaces the gradient in the update by an exponential average of all past gradients including the current one, which allows for a smoother training. The gradient update with momentum becomes:

$$
\begin{split}
    m^{(t)} & = \beta_1 m^{(t-1)} + (1 - \beta_1)\cdot g^{(t)}\\
    w^{(t)} & = w^{(t-1)} - \eta \cdot m^{(t)}\\
\end{split}
$$

Let's implement the optimizer below:

In [ ]:
class SGDMomentum(OptimizerTemplate):
    
    def __init__(self, params: nn.ParameterList, lr: float, momentum: float=0.9)->None:
        super().__init__(params, lr)
        self.momentum = momentum # Corresponds to beta_1 in the equation above
        self.param_momentum = {p: torch.zeros_like(p.data) for p in self.params} # Dict to store m_t
        
    def update_param(self, p:nn.Parameter)->None:
        # TODO: 10 points -- Implement the gradient update
        self.param_momentum[p] = self.momentum*self.param_momentum[p] + (1-self.momentum)*p.grad
        p.data = p.data - self.lr*self.param_momentum[p]        

To verify that our optimizer is working, let's create a challenging surface over two parameter dimensions which we want to optimize to find the optimum:

In [ ]:
def pathological_curve_loss(w1: float, w2: float) -> float:
    # Example of a pathological curvature. There are many more possible, feel free to experiment here!
    x1_loss = torch.tanh(w1)**2 + 0.01 * torch.abs(w1)
    x2_loss = torch.sigmoid(w2)
    return x1_loss + x2_loss

In [ ]:
def plot_curve(curve_fn, x_range: Tuple[float, float] = (-5,5), y_range: Tuple[float, float]=(-5,5), plot_3d: bool=False, cmap=cm.viridis, title: str="Pathological curvature"):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(projection="3d") if plot_3d else fig.add_subplot()

    x = torch.arange(x_range[0], x_range[1], (x_range[1]-x_range[0])/100.)
    y = torch.arange(y_range[0], y_range[1], (y_range[1]-y_range[0])/100.)
    x, y = torch.meshgrid([x,y], indexing='ij')
    z = curve_fn(x, y)
    x, y, z = x.numpy(), y.numpy(), z.numpy()

    if plot_3d:
        ax.plot_surface(x, y, z, cmap=cmap, linewidth=1, color="#000", antialiased=False)
        ax.set_zlabel("loss")
    else:
        ax.imshow(z.T[::-1], cmap=cmap, extent=(x_range[0], x_range[1], y_range[0], y_range[1]))
    plt.title(title)
    ax.set_xlabel(r"$w_1$")
    ax.set_ylabel(r"$w_2$")
    plt.tight_layout()
    return ax

sns.reset_orig()
_ = plot_curve(pathological_curve_loss, plot_3d=True)
plt.show()

In terms of optimization, you can image that $w_1$ and $w_2$ are weight parameters, and the curvature represents the loss surface over the space of $w_1$ and $w_2$. Note that in typical networks, we have many, many more parameters than two, and such curvatures can occur in multi-dimensional spaces as well.

Ideally, our optimization algorithm would find the center of the ravine and focuses on optimizing the parameters towards the direction of $w_2$. However, if we encounter a point along the ridges, the gradient is much greater in $w_1$ than $w_2$, and we might end up jumping from one side to the other. Due to the large gradients, we would have to reduce our learning rate slowing down learning significantly.

To test our algorithms, we can implement a simple function to train two parameters on such a surface:

In [ ]:
def train_curve(optimizer_func, curve_func=pathological_curve_loss, num_updates=100, init=[5,5]):
    """
    Inputs:
        optimizer_func - Constructor of the optimizer to use. Should only take a parameter list
        curve_func - Loss function (e.g. pathological curvature)
        num_updates - Number of updates/steps to take when optimizing 
        init - Initial values of parameters. Must be a list/tuple with two elements representing w_1 and w_2
    Outputs:
        Numpy array of shape [num_updates, 3] with [t,:2] being the parameter values at step t, and [t,2] the loss at t.
    """
    weights = nn.Parameter(torch.FloatTensor(init), requires_grad=True)
    optimizer = optimizer_func([weights])
    
    list_points = []
    for _ in range(num_updates):
        # TODO: 8 points -- Determine the loss for the current weights, save the weights and loss, perform backpropagation
        # Get loss using the curve function
        loss = curve_func(weights[0], weights[1])
        # Zero all gradients
        optimizer.zero_grad()
        list_points.append(torch.tensor([weights[0], weights[1], loss]))
        # Compute gradients and update params
        loss.backward()
        optimizer.step()
        
    points = torch.stack(list_points, dim=0).detach().cpu().numpy()
    return points

Next, let's apply the optimizer on our curvature. Note that we set a much higher learning rate for the optimization algorithms as you would in a standard neural network. This is because we only have 2 parameters instead of tens of thousands or even millions.

In [ ]:
SGDMom_points = train_curve(lambda params: SGDMomentum(params, lr=10, momentum=0.9))
print(SGDMom_points)

To understand best how the different algorithms worked, we visualize the update step as a line plot through the loss surface. We will stick with a 2D representation for readability.

In [ ]:
all_points = SGDMom_points
ax = plot_curve(pathological_curve_loss,
                x_range=(-np.absolute(all_points[:,0]).max(), np.absolute(all_points[:,0]).max()),
                y_range=(all_points[:,1].min(), all_points[:,1].max()),
                plot_3d=False)
ax.plot(SGDMom_points[:,0], SGDMom_points[:,1], color="red", marker="o", zorder=2, label="SGDMom")
plt.legend()
plt.show()

If the implementation is correct, you should see that the optimizer indeed reaches a point of very low $w_2$ ($w_2 < -7.5$) and $w_1\approx 0$. If not, go back to your optimizer implementation and check what could go wrong.

## Part 1. Generic training and validation function

Now that we the optimizer implemented, and the dataset loaded, we can look at implementing our own training functions. 

In [ ]:
def train_one_epoch(model: nn.Module, optimizer: OptimizerTemplate, loss_module, data_loader)->Tuple[float, int]:
    true_preds, count = 0.0, 0
    model.train()
    for imgs, labels in data_loader:
        # TODO: 10 points -- Implement training loop with training on classification
        # Pushing data to device for GPU utilization
        imgs = imgs.to(device)
        labels = labels.to(device)
        # Compute output, loss, gardients and update params
        output = model(imgs)
        optimizer.zero_grad()
        loss = loss_module(output, labels)
        loss.backward()
        optimizer.step()
        # Record statistics during training
        # The index of the maximum in the output is considered the prediction
        _, preds = torch.max(output,1)
        count += labels.size()[0]
        true_lbls = preds==labels
        true_preds += sum(true_lbls).item()
        
    train_acc = true_preds / count
    return train_acc

In [ ]:
@torch.no_grad()
def test_model(model, data_loader):
    # TODO: 10 points - Test model and return accuracy
    true_preds, count = 0.0, 0
    # Model is set in evaluation mode
    model.to(device)
    model.eval()
    for imgs, labels in data_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        output = model(imgs)
        _, preds = torch.max(output,1)
        count += labels.size()[0]
        true_lbls = preds==labels
        true_preds += sum(true_lbls).item()
        
    test_acc = true_preds / count
    return test_acc    

In [ ]:
def save_model(model, model_name, root_dir=CHECKPOINT_PATH):
    # TODO: 2 points -- Save the parameters of the model
    # Create the directory if it does not exist
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
    # Save the model in the given path
    path = os.path.join(root_dir, model_name+'.pth')
    torch.save(model.state_dict(), path)    

def load_model(model, model_name, root_dir=CHECKPOINT_PATH):
    # TODO: 2 points -- Load the parameters of the model
    # Load the model from the given path
    path = os.path.join(root_dir, model_name+'.pth')
    model.load_state_dict(torch.load(path))
    return model

In [ ]:
def train_model(model, optimizer, loss_module, train_data_loader, val_data_loader, num_epochs=25, model_name="MyModel"):
    # Set model to train mode
    model.to(device)
    best_val_acc = -1.0

    # Training loop
    for epoch in range(1, num_epochs+1):
        train_acc = train_one_epoch(model, optimizer, loss_module, train_data_loader)
        
        if epoch % 5 == 0 or epoch == num_epochs:
            # Evaluate the model and save if best
            acc = test_model(model, val_data_loader)
            if acc > best_val_acc:
                best_val_acc = acc 
                save_model(model, model_name, CHECKPOINT_PATH)
            print(
                f"[Epoch {epoch+1:2d}] Training accuracy: {train_acc*100.0:05.2f}%, Validation accuracy: {acc*100.0:05.2f}%, Best validation accuracy: {best_val_acc*100.0:05.2f}%"
            )

    # Load best model after training
    model = load_model(model, model_name, CHECKPOINT_PATH)
    return model 

## Part 2: Implementing the MLP



### Part 2.1: Vanilla MLP Module

You can make use of PyTorch's common functionalities, especially the `torch.nn` modules might be of help. The design choices of the MLP (e.g. the activation function) is left up to you, but for an initial setup, we recommend stacking linear layers with ReLU activation functions in between. Remember to not apply any activation function on the output.

In [ ]:
class MLP(nn.Module):

    def __init__(self, input_dim: int=3072, num_classes: int=10, hidden_dims: List[int]=[256, 128], act_layer: nn.Module=nn.ReLU)->None:
        """
        Inputs:
            input_dim - Dimension of the input images in pixels
            num_classes - Number of classes we want to predict. The output size of the MLP
                          should be num_classes.
            hidden_dims - A list of integers specifying the hidden layer dimensions in the MLP. 
                           The MLP should have len(hidden_sizes)+1 linear layers.
            act_layer - Activation function.
        """
        super().__init__()
        # TODO: 5 points - Create the network based on the specified hidden sizes
        # Store all the layers in a list
        self.layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            self.layers.append(nn.Linear(input_dim, hidden_dims[i], device=device))
            self.layers.append(act_layer())
            input_dim = hidden_dims[i]
        # Add the last output layer 
        self.layers.append(nn.Linear(input_dim, num_classes, device=device))
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # TODO: 5 points -- Apply the MLP on an input
        #x = x.view(x.size(0),-1)
        x = torch.flatten(x,1)
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters()) / 1e6

In [ ]:
# Let's test the MLP implementation
input_dim = np.random.randint(low=64, high=3072)
num_classes = np.random.randint(low=5, high=20)
hidden_dims = [np.random.randint(low=32, high=256) for _ in range(np.random.randint(low=1, high=3))]
my_mlp = MLP(input_dim=input_dim, num_classes=num_classes, hidden_dims=hidden_dims)
my_mlp.to(device)
random_input = torch.randn(32, input_dim, device=device)
random_output = my_mlp(random_input)
assert random_output.shape[0] == random_input.shape[0]
assert random_output.shape[1] == num_classes

In [ ]:
# Create model, optimizer, and start training
model_mlp = MLP(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_mlp.parameters(), lr=0.06) # you may tune lr
loss_module = nn.CrossEntropyLoss().to(device)

print(f'model mlp created: {count_parameters(model_mlp):05.3f}M')
model_mlp = train_model(model_mlp, optimizer, loss_module, train_loader, val_loader, num_epochs=5, model_name="myMLP_ReLU")

In [ ]:
# Test best model on test set
vanilla_mlp_test_acc = test_model(model_mlp, test_loader)
print(f'Test accuracy: {vanilla_mlp_test_acc*100.0:05.2f}%')

It is expected that you reach with the default configurations a validation and test accuracy of $\sim51-53\%$ . If you have reached this performance, we can consider this task as completed!

### Part 2.2: Improving the vanilla MLP

Now that we have a basic MLP, let's try to improve over this default performance! Your task is to think about ways to maximize the performance of your MLP. Possible suggestions you can look at include:
 
* Do data augmentations help the model to generalize?
* Can regularization techniques (dropout, weight decay, etc.) help?
* Do deeper models perform better? Or is it better to have wide networks?
* Can normalization techniques (BatchNorm, LayerNorm, etc.) help?

Your task is to improve your model to reach at least 56% on the test set! But can you get even above this? Consider this as a challenge! 

For this implementation, you can copy and then modify your codes above. List the changes that you have made and discuss what affect they have. Further, repeat the experiments for *at least 3 seeds* to report stable improvements.

In [ ]:
# Seed implementation 1
seed = 42
random_seed(seed=seed, deterministic=True)
train_set, val_set = torch.utils.data.random_split(main_dataset, [45000, 5000], generator=torch.Generator().manual_seed(seed))
train_loader = data.DataLoader(train_set, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=3)
val_loader = data.DataLoader(val_set, batch_size=128, shuffle=False, drop_last=False, num_workers=3)

In [ ]:
def train_model_2(model, optimizer, loss_module, train_data_loader, val_data_loader, num_epochs=25, model_name="MyModel"):
    # Set model to train mode
    model.to(device)
    best_val_acc = -1.0
    tr_acc = []
    val_acc = []
    # Training loop
    for epoch in range(1, num_epochs+1):
        train_acc = train_one_epoch(model, optimizer, loss_module, train_data_loader)
        # Saving accuracies for plots/analysis
        acc = test_model(model, val_data_loader)
        tr_acc.append(train_acc)
        val_acc.append(acc)
        #print(f"[Epoch {epoch+1:2d}] Training accuracy: {train_acc*100.0:05.2f}%, Validation accuracy: {acc*100.0:05.2f}%")
        if epoch % 5 == 0 or epoch == num_epochs:
            # Evaluate the model and save if best
            #acc = test_model(model, val_data_loader)
            if acc > best_val_acc:
                best_val_acc = acc 
                save_model(model, model_name, CHECKPOINT_PATH)
            print(
                f"[Epoch {epoch+1:2d}] Training accuracy: {train_acc*100.0:05.2f}%, Validation accuracy: {acc*100.0:05.2f}%, Best validation accuracy: {best_val_acc*100.0:05.2f}%"
            )

    # Load best model after training
    model = load_model(model, model_name, CHECKPOINT_PATH)
    return model, tr_acc, val_acc

In [ ]:
class MLP_2(nn.Module):

    def __init__(self, input_dim: int=3072, num_classes: int=10, hidden_dims: List[int]=[512, 256, 128], act_layer: nn.Module=nn.ReLU)->None:
        super().__init__()
        # TODO: 5 points - Create the network based on the specified hidden sizes
        # Adding batch norm layers to normalize ldayers and maintain consistent distribution
        # Dropout layers to avoid overfitting on training
        self.layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            self.layers.append(nn.Linear(input_dim, hidden_dims[i], device=device))
            self.layers.append(nn.BatchNorm1d(hidden_dims[i]))
            self.layers.append(act_layer())
            self.layers.append(nn.Dropout(p=0.3))
            input_dim = hidden_dims[i]

        self.layers.append(nn.Linear(input_dim, num_classes, device=device))
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # TODO: 5 points -- Apply the MLP on an input
        #x = x.view(x.size(0),-1)
        x = torch.flatten(x,1)
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
# Create model, optimizer, and start training
model_mlp_2 = MLP_2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_mlp_2.parameters(), lr=0.05) # you may tune lr
loss_module = nn.CrossEntropyLoss().to(device)

print(f'model mlp 2.2 created: {count_parameters(model_mlp_2):05.3f}M')
model_mlp_2, tr_acc, val_acc=train_model_2(model_mlp_2, optimizer, loss_module, train_loader, val_loader, num_epochs=25, model_name="myMLP_ReLU")

In [ ]:
# Test best model on test set
vanilla_mlp_2_test_acc = test_model(model_mlp_2, test_loader)
print(f'Test accuracy: {vanilla_mlp_2_test_acc*100.0:05.2f}%')

In [ ]:
# Seed implementation 2
seed = 123
random_seed(seed=seed, deterministic=True)
train_set, val_set = torch.utils.data.random_split(main_dataset, [45000, 5000], generator=torch.Generator().manual_seed(seed))
train_loader = data.DataLoader(train_set, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=3)
val_loader = data.DataLoader(val_set, batch_size=128, shuffle=False, drop_last=False, num_workers=3)

In [ ]:
# Create model, optimizer, and start training
model_mlp_2 = MLP_2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_mlp_2.parameters(), lr=0.05) # you may tune lr
loss_module = nn.CrossEntropyLoss().to(device)

print(f'model mlp 2.2 created: {count_parameters(model_mlp_2):05.3f}M')
model_mlp_2, tr_acc, val_acc=train_model_2(model_mlp_2, optimizer, loss_module, train_loader, val_loader, num_epochs=25, model_name="myMLP_ReLU")

In [ ]:
# Test best model on test set
vanilla_mlp_2_test_acc = test_model(model_mlp_2, test_loader)
print(f'Test accuracy: {vanilla_mlp_2_test_acc*100.0:05.2f}%')

In [ ]:
# Seed implementation 3
seed = 88
random_seed(seed=seed, deterministic=True)
train_set, val_set = torch.utils.data.random_split(main_dataset, [45000, 5000], generator=torch.Generator().manual_seed(seed))
train_loader = data.DataLoader(train_set, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=3)
val_loader = data.DataLoader(val_set, batch_size=128, shuffle=False, drop_last=False, num_workers=3)

In [ ]:
# Create model, optimizer, and start training
model_mlp_2 = MLP_2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_mlp_2.parameters(), lr=0.05) # you may tune lr
loss_module = nn.CrossEntropyLoss().to(device)

print(f'model mlp 2.2 created: {count_parameters(model_mlp_2):05.3f}M')
model_mlp_2, tr_acc, val_acc=train_model_2(model_mlp_2, optimizer, loss_module, train_loader, val_loader, num_epochs=25, model_name="myMLP_ReLU")

In [ ]:
# Test best model on test set
vanilla_mlp_2_test_acc = test_model(model_mlp_2, test_loader)
print(f'Test accuracy: {vanilla_mlp_2_test_acc*100.0:05.2f}%')

In [ ]:
# TODO: 10 points -- Improve the model and list the changes as code comments. 
# You may try different changes (e.g. those suggested above) individually or combine them. 
# You do not need to explore all the four suggestions above.
# You can create as many code blocks as you need. 
#raise NotImplementedError

#### Observation:
I have implemented the regularization and normalization technique in this model. Added batch norm layers to normalize layers and maintain consistent distribution throughout the network. And Dropout layers to avoid overfitting on training data so it can increase the validation accuracy and better generalize.

### Part 2.3: Investigating the limitations of MLP

Now that we have seen how we can optimize our MLP, it is good to investigate the limitations of the model as well. Images have a natural grid structure where close-by pixels are strongly related. Does the MLP make use of this structure? To investigate this question, we will run two experiments:
 
* Create a shuffle of pixels at the beginning of the training, and use the same shuffle throughout the training and validation.
* At each training and validation step, sample a new shuffle of pixels.
 
 
It is up to you whether you perform this investigation on the original plain MLP version or your optimized one. Implement a corresponding train and test function that support both variants of shuffling, and train two models.

In [ ]:
# TODO: 2 points - create a function that shuffles pixel values of images
def shuffle_pixels(imgs: torch.Tensor, shuffle_idx_shared: Optional[torch.Tensor] = None) -> torch.Tensor:
    B, C, H, W = imgs.shape
    if shuffle_idx_shared is not None:
        # shuffle the pixels using the provided shuffle idx
        imgs = imgs.view(B,C,-1)[:,:,shuffle_idx_shared]
    else:
        # Sample a shuffle idx and then shuffle the pixels
        shuffle_idx = torch.randperm(H*W)
        imgs = imgs.view(B,C,-1)[:,:,shuffle_idx]
        
    return imgs.view(B,C,H,W)

In [ ]:
# visualized the effects of pixel shuffle
images_pixelshuffled = shuffle_pixels(images)
print(images_pixelshuffled.shape)
imshow(torchvision.utils.make_grid(images_pixelshuffled))
print("GroundTruth (1st row): ", " ".join(f"{classes[labels[j]]:5s}" for j in range(8)))

In [ ]:
# Create a training and test function that supports the shuffling of pixels, either fixed and shared among the entire dataset or newly generated for each batch

def train_one_epoch_pixelshuffled(
    model: nn.Module,
    optimizer: OptimizerTemplate,
    loss_module,
    data_loader,
    shuffle_idx_shared: Optional[torch.Tensor] = None,
) -> Tuple[float, int]:
    true_preds, count = 0.0, 0
    model.train()
    for imgs, labels in data_loader:
        # TODO: 2 points - Implement training loop with training on classification
        # Call the function to shuffle pixels
        imgs = shuffle_pixels(imgs, shuffle_idx_shared)
        # Same process as before
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(imgs)
        loss = loss_module(output, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(output,1)
        # Record statistics during training
        count += labels.size()[0]
        true_lbls = preds==labels
        true_preds += sum(true_lbls).item()
        
    train_acc = true_preds / count
    return train_acc

In [ ]:
@torch.no_grad()
def test_model_pixelshuffled(
    model: nn.Module,
    data_loader,
    shuffle_idx_shared: Optional[torch.Tensor] = None,
) -> float:
    # TODO: 2 points -- Test model and return accuracy
    # Set to evaluation mode
    model.eval()
    true_preds, count = 0.0, 0
    for imgs, labels in data_loader:
        imgs = shuffle_pixels(imgs, shuffle_idx_shared)
        imgs, labels = imgs.to(device), labels.to(device)
        output = model(imgs)
        _, preds = torch.max(output,1)
        # Record statistics during training
        count += labels.size()[0]
        true_lbls = preds==labels
        true_preds += sum(true_lbls).item()
        
    test_acc = true_preds / count
    return test_acc

In [ ]:
def train_model_pixelshuffled(
    model,
    optimizer,
    loss_module,
    train_data_loader,
    val_data_loader,
    num_epochs=25,
    model_name="MyMLP_pixelshuffled",
    shuffle_idx_shared: Optional[torch.Tensor] = None,
):
    # Set model to train mode
    model.to(device)
    best_val_acc = -1.0

    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_acc = train_one_epoch_pixelshuffled(model, optimizer, loss_module, train_data_loader, shuffle_idx_shared=shuffle_idx_shared)

        if epoch % 5 == 0 or epoch == num_epochs:
            # Evaluate the model and save if best
            acc = test_model_pixelshuffled(model, val_data_loader, shuffle_idx_shared=shuffle_idx_shared)
            if acc > best_val_acc:
                best_val_acc = acc
                save_model(model, model_name, CHECKPOINT_PATH)

            print(
                f"[Epoch {epoch+1:2d}] Training accuracy: {train_acc*100.0:05.2f}%, Validation accuracy: {acc*100.0:05.2f}%, Best validation accuracy: {best_val_acc*100.0:05.2f}%"
            )

    # Load best model after training
    model = load_model(model, model_name, CHECKPOINT_PATH)
    return model

In [ ]:
# Create model, optimizer, and start training on fixed shuffling of pixels
model_mlp_pixelshuffled_shared = MLP(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_mlp_pixelshuffled_shared.parameters(), lr=0.1)
loss_module = nn.CrossEntropyLoss().to(device)

# TODO: 2 points -- Create a shared: shuffle_idx
# Generate a random shuffle index tensor
_,_, H, W = images.shape
shuffle_idx = torch.randperm(H*W)

model_mlp_pixelshuffled_shared = train_model_pixelshuffled(
    model_mlp_pixelshuffled_shared,
    optimizer,
    loss_module,
    train_loader,
    val_loader,
    num_epochs=5,
    model_name="myMLP_ReLU_pixelshuffled_shared",
    shuffle_idx_shared=shuffle_idx,
)

In [ ]:
# Create model, optimizer, and start training on a new shuffling of pixels per batch/image
model_mlp_pixelshuffled_batch = MLP(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_mlp_pixelshuffled_batch.parameters(), lr=0.1)
loss_module = nn.CrossEntropyLoss().to(device)

model_mlp_pixelshuffled_batch = train_model_pixelshuffled(
    model_mlp_pixelshuffled_batch,
    optimizer,
    loss_module,
    train_loader,
    val_loader,
    num_epochs=5,
    model_name="myMLP_ReLU_pixelshuffled_batch",
    shuffle_idx_shared=None,
)

What results do you observe? What does this tell us about the MLP being aware of the image structure? Add your results and observations below.

Observation: The accuracy of the model with the fixed shuffling of pixels is higher than the dynamic shuffling of pixels. This is because learning the consistent patterns across all image data is easier than the dynamic pixel shuffling performed for each training/validation step.

Dynamic shuffling requires the model to learn features that are invariant to changes in the spatial arrangement of pixels and it encourages the model to learn more robust and generalizable features. This requires the model to capture spatial data which is not possible with MLPs. For relatively simple tasks or datasets, MLPs may learn some basic features, but they may struggle with more intricate spatial relationships.

Since the MLPs could not capture the spatial information of the image data, they perform poorly when dynamic shuffling of pixels is done than a fixed shuffling of pixels as the data for fixed shuffling of pixels has consistent spatial patterns.

## Part 3: Implementing the ConvNet

We will repeat the experiments in Part 2, but with a ConvNet.

### Part 3.1: Vanilla ConvNet

Let's start with a simple LeNet like ConvNet, consisting of a number of convolution blocks (e.g., two Conv+ReLU+MaxPool blocks) followed by a MLP consisting of two hidden layers.

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, in_chans: int=3, base_dims: int=32, num_classes:int=10, act_layer=nn.ReLU)->None:
        super().__init__()
        # TODO: 2 points -- write code for two Conv+ReLU+MaxPool blocks
        # Sequential block with conv+ReLU+MaxPool layers
        self.conv = nn.Sequential(
            nn.Conv2d(in_chans, base_dims, kernel_size=3),
            act_layer(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(base_dims, base_dims*2, kernel_size=3),
            act_layer(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Dense layers after the conv network 
        self.mlp = MLP(input_dim=base_dims*2, num_classes=num_classes, hidden_dims=[base_dims*4, base_dims*2], act_layer=act_layer)

    def forward(self, x):
        x = self.conv(x)
        # Perform global average pooling 
        x = x.mean(dim=(2, 3))
        x = self.mlp(x)
        return x

In [ ]:
# Create model, optimizer, and start training

model_convnet = ConvNet(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_convnet.parameters(), lr=0.1)
loss_module = nn.CrossEntropyLoss().to(device)

print(f"model convnet created: {count_parameters(model_convnet):05.3f}M")
model_convnet = train_model(
    model_convnet,
    optimizer,
    loss_module,
    train_loader,
    val_loader,
    num_epochs=5,
    model_name="myConvNet_ReLU",
)

In [ ]:
# Test best model on test set
vanilla_convnet_test_acc = test_model(model_convnet, test_loader)
print(f"Test accuracy: {vanilla_convnet_test_acc*100.0:05.2f}%")

You will observe accuracy around $\sim41-43\%$, worse than the vanilla MLP. Next, we will try to address the performance issue by modifying the specifications of ConvNets.

### Part 3.2: Improving the ConvNet

Similar to Part 2.2. Please explore how to redesign your ConvNet to improve the performance.

In [ ]:
# TODO: 10 points -- Improve the model and list the changes as code comments.
# You may try different changes (e.g. those suggested in Part 2.2) individually or combine them.
# You do not need to explore all the four suggestions above.
# You can create as many code blocks as you need.
# raise NotImplementedError

In [ ]:
class ConvNet_ver2(nn.Module):
    def __init__(self, in_chans: int=3, base_dims: int=32, num_classes:int=10, act_layer=nn.ReLU)->None:
        super().__init__()
        # TODO: 2 points -- write code for two Conv+ReLU+MaxPool blocks
        # Adding batch norm layers to normalize ldayers and maintain consistent distribution
        # Dropout layers to avoid overfitting on training
        self.conv = nn.Sequential(
            nn.Conv2d(in_chans, base_dims, kernel_size=3, padding='same'),
            nn.BatchNorm2d(base_dims),
            act_layer(),
            nn.Conv2d(base_dims, base_dims, kernel_size=3, padding='same'),
            nn.BatchNorm2d(base_dims),
            act_layer(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),

            nn.Conv2d(base_dims, base_dims*2, kernel_size=3, padding='same'),
            nn.BatchNorm2d(base_dims*2),
            act_layer(),
            nn.Conv2d(base_dims*2, base_dims*2, kernel_size=3, padding='same'),
            nn.BatchNorm2d(base_dims*2),
            act_layer(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),

            nn.Conv2d(base_dims*2, base_dims*4, kernel_size=3, padding='same'),
            nn.BatchNorm2d(base_dims*4),
            act_layer(),
            nn.Conv2d(base_dims*4, base_dims*4, kernel_size=3, padding='same'),
            nn.BatchNorm2d(base_dims*4),
            act_layer(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),
            
        )
        self.mlp = nn.Sequential(
            nn.Linear(base_dims*4, base_dims*4),
            #nn.Linear(2048, base_dims*4),
            act_layer(),
            nn.Dropout(0.3),
            nn.Linear(base_dims*4, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.mean(dim=(2, 3))
        #x = torch.flatten(x,1)
        x = self.mlp(x)
        return x

In [ ]:
# Create model, optimizer, and start training

model_convnet_ver2 = ConvNet_ver2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_convnet_ver2.parameters(), lr=0.1)
loss_module = nn.CrossEntropyLoss().to(device)

print(f"model convnet version 2 created: {count_parameters(model_convnet_ver2):05.3f}M")
model_convnet_ver2 = train_model(
    model_convnet_ver2,
    optimizer,
    loss_module,
    train_loader,
    val_loader,
    num_epochs=15,
    model_name="myConvNet_ver2_ReLU",
)

In [ ]:
# Test best model on test set
vanilla_convnet_test_acc = test_model(model_convnet_ver2, test_loader)
print(f"Test accuracy: {vanilla_convnet_test_acc*100.0:05.2f}%")

#### Observation:

ConvNets perform well on image data as it can capture the spatial information of the image data. We flattened the image into 1 dimension in the initial step of MLP. This led to loss of information as we won’t be able to know if the pixels were connected to each other or not. 

### Part 3.3: Testing if ConvNets can address the limitations of MLP

Similar to Part 2.3. Test your improved ConvNet to compare the results with those in Part 2.3.

In [ ]:
# TODO: 2 points -- Train your best ConvNet using data with a shared pixel shuffle and check the accuracy for training, validation and testing datasets
# Create model, optimizer, and start training on fixed shuffling of pixels
model_conv_pixelshuffled_shared = ConvNet_ver2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_conv_pixelshuffled_shared.parameters(), lr=0.1)
loss_module = nn.CrossEntropyLoss().to(device)

# TODO: 2 points -- Create a shared: shuffle_idx
# Generate a random shuffle index tensor
_,_, H, W = images.shape
shuffle_idx = torch.randperm(H*W)

model_conv_pixelshuffled_shared = train_model_pixelshuffled(
    model_conv_pixelshuffled_shared,
    optimizer,
    loss_module,
    train_loader,
    val_loader,
    num_epochs=5,
    model_name="myCONV_ReLU_pixelshuffled_shared",
    shuffle_idx_shared=shuffle_idx,
)

In [ ]:
# TODO: 2 points -- Train your best ConvNet using data with a # TODO: Train your best ConvNet using data with a per-batch pixel shuffle and check the accuracy for training, validation and testing datasets pixel shuffle and check the accuracy for training, validation and testing datasets
# Create model, optimizer, and start training on a new shuffling of pixels per batch/image
model_conv_pixelshuffled_shared = ConvNet_ver2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_conv_pixelshuffled_shared.parameters(), lr=0.1)
loss_module = nn.CrossEntropyLoss().to(device)

model_conv_pixelshuffled_shared = train_model_pixelshuffled(
    model_conv_pixelshuffled_shared,
    optimizer,
    loss_module,
    train_loader,
    val_loader,
    num_epochs=5,
    model_name="myCONV_ReLU_pixelshuffled_batch",
    shuffle_idx_shared=None,
)

#### Observation:
MLP performed well with the given models on fixed shuffling of pixels. This suggests that while MLPs may not inherently capture spatial structures as well as ConvNets, they can still learn and generalize better with augmented data.

The ConvNet's performance with dynamic shuffling is closer to the fixed shuffling result. ConvNets are more adept at capturing spatial hierarchies and local patterns, and the increased variability from dynamic shuffling has a less detrimental effect on their performance compared to MLPs.

## Part 4: Fitting Noise

We will re-create a training dataset with ground-truth labels shuffled (i.e., labels become noises). Then, we train our MLPs and ConvNets to check the training accuracy. If they could do a good job, it means that there are potential risks of fitting noises of deep neural networks, and we shall be careful in monitoring the data quality in supervised training.

### Shuffle the Ground-Truth Labels

In [ ]:
# TODO: 2 points -- Create a training dataset with ground-truth labels shuffled
# Extract labels from the training set
original_train_labels = torch.tensor(train_set.dataset.targets)

# Shuffle the labels to create noise
shuffled_labels = original_train_labels[torch.randperm(len(original_train_labels))]

# Update the labels in the training set with shuffled labels
train_set.dataset.targets = shuffled_labels.tolist()

# Use noise_train_dataset to store the dataset with shuffled labels
noise_train_dataset = train_set

# Create data loaders for later
noise_train_loader = data.DataLoader(
    noise_train_dataset,
    batch_size=128,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
    num_workers=3,
)


### Verify labels are shuffled

In [ ]:
noise_dataiter = iter(noise_train_loader)
noise_images, noisey_labels = next(noise_dataiter)

In [ ]:
imshow(torchvision.utils.make_grid(noise_images))
for i in range(0, len(noisey_labels), 8):
    print(
        f"Noisy labels (row {i//8}): ",
        " ".join(f"{classes[noisey_labels[i+j]]:5s}" for j in range(8)),
    )

### Training MLPs and ConvNets on the noisy datasets

In [ ]:
# TODO: 5 points -- Train your best MLP and check the training accuracy
# Create model, optimizer, and start training
model_mlp_2 = MLP_2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_mlp_2.parameters(), lr=0.05) # you may tune lr
loss_module = nn.CrossEntropyLoss().to(device)

print(f'model mlp 2.2 created: {count_parameters(model_mlp_2):05.3f}M')
model_mlp_2, tr_acc, val_acc=train_model_2(model_mlp_2, optimizer, loss_module, noise_train_loader, val_loader, num_epochs=25, model_name="myMLP_ReLU")

# Test best model on test set
vanilla_mlp_2_test_acc = test_model(model_mlp_2, test_loader)
print(f'Test accuracy: {vanilla_mlp_2_test_acc*100.0:05.2f}%')

In [ ]:
# TODO: 5 points -- Train your best ConvNet and check the training accuracy
# Create model, optimizer, and start training

model_convnet_ver2 = ConvNet_ver2(act_layer=nn.ReLU).to(device)
optimizer = SGDMomentum(model_convnet_ver2.parameters(), lr=0.1)
loss_module = nn.CrossEntropyLoss().to(device)

print(f"model convnet version 2 created: {count_parameters(model_convnet_ver2):05.3f}M")
model_convnet_ver2 = train_model(
    model_convnet_ver2,
    optimizer,
    loss_module,
    noise_train_loader,
    val_loader,
    num_epochs=15,
    model_name="myConvNet_ver2_ReLU",
)

In [ ]:
# Test best model on test set
vanilla_convnet_test_acc = test_model(model_convnet_ver2, test_loader)
print(f"Test accuracy: {vanilla_convnet_test_acc*100.0:05.2f}%")

#### Observation:

The results show the significance of monitoring and ensuring the quality of training data. Noisy or mislabeled data can significantly impact model performance, and strategies for data quality control are crucial.

## Conclusion

You have reached the end of the practical, congratulations! Now, you should have a good idea of what it means to train a MLP, how we can use neural networks to perform image classification, what aspects there are to improve a networks performance, and what limitations and risks there are. 